# LightGBM でバッチ推論を行う
## 0. はじめに
カスタムコンテナで LightGBM のバッチ推論をするサンプルノートブックです。モデルの学習とリアルタイム推論については [こちらのノートブック（英語）](https://github.com/aws-samples/amazon-sagemaker-script-mode/blob/master/lightgbm-byo/lightgbm-byo.ipynb) をご参照ください。

このノートブックでは、Transform API を使ったバッチ推論と、Amazon SageMaker Processing を使ったバッチ推論の方法をご紹介します。

In [ ]:
# create directory for inference sources
!mkdir -p docker-inference-transform

In [ ]:
%matplotlib inline

import boto3
import sys
import sagemaker
import numpy as np
from sagemaker import get_execution_role

role = get_execution_role()
region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
session = sagemaker.Session()
s3_output = session.default_bucket()
s3_prefix = 'lightGBM-BYO'
tag = ':latest'

## 1. Transform API を使用してバッチ推論する
### 1-1. 推論用スクリプトの準備

[Multi Model Server (MMS)](https://github.com/awslabs/multi-model-server) を使ってバッチ推論を行います。MMS は ModelHandler のなかでデータの前処理を行い、学習済みモデルを使って推論を行い、出力の後処理を行います。以下は inferennce script です。

In [ ]:
%%writefile docker-inference-transform/model_script.py

from collections import namedtuple
import glob
import json
import logging
import os
import re

import lightgbm as lgb
import numpy as np
from sagemaker_inference import content_types, encoder

NUM_FEATURES = 12

class ModelHandler(object):
    """
    A lightGBM Model handler implementation.
    """

    def __init__(self):
        self.initialized = False
        self.model = None

    def initialize(self, context):
        """
        Initialize model. This will be called during model loading time
        :param context: Initial context contains model server system properties.
        :return: None
        """
        self.initialized = True
        properties = context.system_properties
        model_dir = properties.get("model_dir") 
        self.model = lgb.Booster(model_file=os.path.join(model_dir,'online_shoppers_model.txt'))

    def preprocess(self, request):
        """
        Transform raw input into model input data.
        :param request: list of raw requests
        :return: list of preprocessed model input data
        """        
        payload = request[0]['body']
        data = payload.decode('utf-8').splitlines()
        data = np.array(list(map(lambda a:list(map(float,a.split(','))), data)))
        return data

    def inference(self, model_input):
        """
        Internal inference methods
        :param model_input: transformed model input data list
        :return: list of inference output in numpy array
        """
        prediction = self.model.predict(model_input)
        return prediction

    def postprocess(self, inference_output):
        """
        Post processing step - converts predictions to str
        :param inference_output: predictions as numpy
        :return: list of inference output as string
        """

        return [str(inference_output.tolist())]
        
    def handle(self, data, context):
        """
        Call preprocess, inference and post-process functions
        :param data: input data
        :param context: mms context
        """
        
        model_input = self.preprocess(data)
        model_out = self.inference(model_input)
        return self.postprocess(model_out)

_service = ModelHandler()


def handle(data, context):
    if not _service.initialized:
        _service.initialize(context)

    if data is None:
        return None

    return _service.handle(data, context)


## 1-2. MMS を開始するためのスクリプトの準備

以下のスクリプトでは inference toolkit を import し、`model_server.start_model_server` 関数を呼び出して MMS を開始します。この関数は先ほど作成した inference script をモデルサーバに渡します。

In [ ]:
%%writefile docker-inference-transform/dockerd-entrypoint.py

import subprocess
import sys
import shlex
import os
from retrying import retry
from subprocess import CalledProcessError
from sagemaker_inference import model_server

def _retry_if_error(exception):
    return isinstance(exception, CalledProcessError or OSError)

@retry(stop_max_delay=1000 * 50,
       retry_on_exception=_retry_if_error)
def _start_mms():
    # by default the number of workers per model is 1, but we can configure it through the
    # environment variable below if desired.
    # os.environ['SAGEMAKER_MODEL_SERVER_WORKERS'] = '2'
    model_server.start_model_server(handler_service='/home/model-server/model_script.py:handle')

def main():
    if sys.argv[1] == 'serve':
        _start_mms()
    else:
        subprocess.check_call(shlex.split(' '.join(sys.argv[1:])))

    # prevent docker exit
    subprocess.call(['tail', '-f', '/dev/null'])
    
main()

## 1-3. 推論用コンテナの作成

必要なライブラリと先ほど作成した 2つのスクリプトを含むコンテナを作成します。

In [ ]:
%%writefile docker-inference-transform/Dockerfile

FROM ubuntu:18.04
    
# Set a docker label to advertise multi-model support on the container
LABEL com.amazonaws.sagemaker.capabilities.multi-models=false
# Set a docker label to enable container to use SAGEMAKER_BIND_TO_PORT environment variable if present
LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=true

# Install necessary dependencies for MMS and SageMaker Inference Toolkit
RUN apt-get update && \
    apt-get -y install --no-install-recommends \
    build-essential \
    ca-certificates \
    openjdk-8-jdk-headless \
    python3-dev \
    curl \
    vim \
    && rm -rf /var/lib/apt/lists/* \
    && curl -O https://bootstrap.pypa.io/get-pip.py \
    && python3 get-pip.py

RUN update-alternatives --install /usr/bin/python python /usr/bin/python3 1
RUN update-alternatives --install /usr/local/bin/pip pip /usr/local/bin/pip3 1
    
RUN pip install lightgbm numpy pandas \ 
                scikit-learn multi-model-server \
                sagemaker-inference retrying

# Copy entrypoint script to the image
COPY dockerd-entrypoint.py /usr/local/bin/dockerd-entrypoint.py
RUN chmod +x /usr/local/bin/dockerd-entrypoint.py

RUN mkdir -p /home/model-server/

# Copy the default custom service file to handle incoming data and inference requests
COPY model_script.py /home/model-server/model_script.py

# Define an entrypoint script for the docker image
ENTRYPOINT ["python", "/usr/local/bin/dockerd-entrypoint.py"]

# Define command to be passed to the entrypoint
CMD ["serve"]

In [ ]:
ecr_repository_inference = 'lightgbm-byo-inference'
uri_suffix = 'amazonaws.com'
inference_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository_inference + tag)

# Create ECR repository and push docker image
!docker build -t $ecr_repository_inference docker-inference-transform
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository_inference
!docker tag {ecr_repository_inference + tag} $inference_repository_uri
!docker push $inference_repository_uri

### 1-4. バッチ推論の実行

ここでは、学習済みモデルの `model.tar.gz` を使ってバッチ推論を行います。以下の `model_artifact` に、学習済みモデルの S3 パスを設定してください。`input_data` に入力データが格納された S3 パスを設定してください。

In [ ]:
model_artifact =  's3://bucket/lightgbm/model.tar.gz'
input_data = 's3://bucket/lightGBM-BYO/data/test/csv/'

In [ ]:
from sagemaker.transformer import Transformer
from sagemaker import Model, Predictor

model_name = 'lightgbm-byo-deployment'

lgbm_model = Model(model_data=model_artifact,
                   role=role,
                   image_uri=inference_repository_uri,
                   name=model_name)

lgb_transformer = Transformer(model_name=model_name, instance_count=1, instance_type='ml.c5.xlarge')

Transform API を実行してバッチ推論を開始します。SageMaker コンソールの左側のメニューから 推論 -> バッチ変換ジョブ を選択すると、実行したジョブの詳細を確認することができます。

In [ ]:
lgb_transformer.transform(input_data)

## 2. Amazon SageMaker Processing を使用してバッチ推論する
SageMaker Processing の機能を使ってバッチ推論を実現することもできます。既存の推論用カスタムコンテナがある場合はこちらの方法の方がコードの変更が少ないことが多いです。

In [ ]:
!mkdir docker-inference-processing

### 2-1. カスタムコンテナの作成
Dockerfile を作成し、カスタムコンテナをビルドして Amazon ECR に push します。

In [ ]:
%%writefile docker-inference-processing/Dockerfile

FROM ubuntu:18.04

RUN apt-get update && \
    apt-get -y install --no-install-recommends \
    build-essential \
    ca-certificates \
    python3-dev \
    curl \
    vim \
    && rm -rf /var/lib/apt/lists/* \
    && curl -O https://bootstrap.pypa.io/get-pip.py \
    && python3 get-pip.py

    
RUN pip install lightgbm numpy pandas scikit-learn

ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]

In [ ]:
ecr_repository_inference = 'lightgbm-byo-batch-inference'
uri_suffix = 'amazonaws.com'
inference_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository_inference + tag)

# Create ECR repository and push docker image
!docker build -t $ecr_repository_inference docker-inference-processing
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository_inference
!docker tag {ecr_repository_inference + tag} $inference_repository_uri
!docker push $inference_repository_uri

### 2-1. SageMaker Processing の準備
SageMaker Processing のインスタンスのどこに各ファイルを置くかを設定するためのパスを定義します。

In [ ]:
processing_input_dir = '/opt/ml/processing/input'
processing_model_dir = '/opt/ml/processing/model'
processing_output_dir = '/opt/ml/processing/output'
job_name = f'sagemaker-lightgbm'

In [ ]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
processor = ScriptProcessor(base_job_name=job_name,
                                   image_uri=inference_repository_uri,
                                   command=['python3'],
                                   role=role,
                                   instance_count=1,
                                   instance_type='ml.c5.xlarge'
                                  )

### 2-3. バッチ推論用スクリプトの作成
学習済みモデルを使ってバッチ推論するスクリプトを作成します。

In [ ]:
%%writefile batch-inference.py

from collections import namedtuple
import glob
import json
import logging
import os
import re

import lightgbm as lgb
import numpy as np
import argparse


if __name__=='__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--input-dir', type=str, default=None)
    parser.add_argument('--model-dir', type=str, default=None)
    parser.add_argument('--output-dir', type=str, default=None)
    
    args, _ = parser.parse_known_args()
    
    
    filelist = glob.glob(os.path.join(args.input_dir, '*.csv'))
    print(filelist)
    
    model_dir = args.model_dir
    model = lgb.Booster(model_file=os.path.join(model_dir,'online_shoppers_model.txt'))
    
    for f in filelist:
        data = np.loadtxt(f, delimiter=',')
        prediction = model.predict(data)
        np.savetxt(os.path.join(args.output_dir, os.path.basename(f)+'.out'), prediction, fmt='%f')
    

### 2-4. SageMaker Processing ジョブの実行
`model_artifact` に学習済みモデルが保存されている S3 パスを。`inference_data_s3_path` に入力データが保存されている S3 パスを設定して Processing Job を開始します。SageMaker Processing を使ってバッチ推論をする場合は、学習済みモデルを `model.tar.gz` に固める必要はありません。処理結果は `processing_output_dir` に保存されます。

SageMaker コンソールの左側のメニューから 処理中 -> ジョブの処理 を選択すると、実行したジョブの詳細を確認することができます。

In [ ]:
model_artifact = 's3://bucket/lightgbm/online_shoppers_model.txt'
inference_data_s3_path = 's3://bucket/lightGBM-BYO/data/test/csv/'

processor.run(code='./batch-inference.py', # S3 の URI でも可
                     inputs=[ProcessingInput(source=inference_data_s3_path,
                                        destination=processing_input_dir),
                                    ProcessingInput(source=model_artifact,
                                        destination=processing_model_dir)],
                     outputs=[
                         ProcessingOutput(output_name='batch',source=processing_output_dir)],
                      arguments=[
                          '--input-dir',processing_input_dir,
                          '--model-dir',processing_model_dir,
                          '--output-dir',processing_output_dir
                      ]
                    )

## 3. おわりに
以上、2種類のバッチ推論方法をご紹介しました。ご紹介したどちらの方法もすべての入力データに対する推論が完了したら自動的に使用したインスタンスは停止されるため、明示的にインスタンスの停止や削除を行う必要はありません。